In [7]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img, img_to_array
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder
from sqlalchemy import create_engine, Column, Integer, String, LargeBinary
from sqlalchemy.orm import sessionmaker, declarative_base
import os
from sqlalchemy.orm import declarative_base

In [8]:
train_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\trenyravimas"
val_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\validacija"
test_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\testas"

In [9]:

Bazine_klase = declarative_base()
rysys_su_baze = create_engine('sqlite:///pomidoru_lapai.db')

Session = sessionmaker(bind=rysys_su_baze)
sesija = Session()

In [10]:
class PomidoraiTrenyravimas(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_trenyravimo_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)


In [11]:
def irasyti_trenyravimo_paveikslelius(sesija, trenyravimo_kelias):

    for klases_pavadinimas in os.listdir(trenyravimo_kelias):
        klases_kelias = os.path.join(trenyravimo_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTrenyravimas).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTrenyravimas(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti trenyravimo paveiksleiai")

In [12]:
class PomidoraiValidacija(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_validacijos_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

def irasyti_validacijos_paveikslelius(sesija, validacijos_kelias):

    for klases_pavadinimas in os.listdir(validacijos_kelias):
        klases_kelias = os.path.join(validacijos_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiValidacija).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiValidacija(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti validacijos paveiksleiai")

In [13]:
class PomidoraiTestas(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_testo_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

def irasyti_testo_paveikslelius(sesija, testo_kelias):

    for klases_pavadinimas in os.listdir(test_kelias):
        klases_kelias = os.path.join(test_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTestas).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTestas(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti testo paveiksleiai")

In [14]:
Bazine_klase.metadata.create_all(rysys_su_baze)

irasyti_trenyravimo_paveikslelius(sesija,train_kelias)
irasyti_validacijos_paveikslelius(sesija,val_kelias)
irasyti_testo_paveikslelius(sesija,test_kelias)

Irasyti trenyravimo paveiksleiai
Irasyti validacijos paveiksleiai
Irasyti testo paveiksleiai


In [15]:

train_df = pd.read_sql_table('Pomidoru_lapai_trenyravimo_duomenys', con=rysys_su_baze)

val_df = pd.read_sql_table('Pomidoru_lapai_validacijos_duomenys', con=rysys_su_baze)

test_df = pd.read_sql_table('Pomidoru_lapai_testo_duomenys', con=rysys_su_baze)


# print(train_df.head())
# print(val_df.head())
# print(test_df.head())

In [16]:
def issitraukti_paveikslelius(df, dydis=(224, 224)):
    paveiksleliai = []
    klasifikacijos = []

    for indeksas, eilute in df.iterrows():
        paveikslelis = load_img(eilute['kelias'], target_size=dydis)
        paveikslelis_array = img_to_array(paveikslelis)
        paveiksleliai.append(paveikslelis_array)
        klasifikacijos.append(eilute['klases_pavadinimas'])

    return np.array(paveiksleliai), np.array(klasifikacijos)

x_train, y_train = issitraukti_paveikslelius(train_df)
x_val, y_val = issitraukti_paveikslelius(val_df)
x_test, y_test = issitraukti_paveikslelius(test_df)

In [17]:
def uzkoduoti_klases_lable(y_train, y_val, y_test):

    enkoderis = LabelEncoder()
    y_train_skaiciais = enkoderis.fit_transform(y_train)
    y_val_skaiciais = enkoderis.transform(y_val)
    y_test_skaiciais = enkoderis.transform(y_test)

    print("Klases uzkoduotos")

    return y_train_skaiciais,y_val_skaiciais,y_test_skaiciais

y_train, y_val, y_test = uzkoduoti_klases_lable(y_train,y_val,y_test)

Klases uzkoduotos


HSV paveiksleliai
.


In [54]:
import os
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf

In [ ]:
def konvertuoti_i_hsv(pav_failo_kelias):
    paveikslelis = cv2.imread(pav_failo_kelias)
    paveikslelis = cv2.cvtColor(paveikslelis, cv2.COLOR_BGR2RGB)
    hsv_paveikslelis = cv2.cvtColor(paveikslelis, cv2.COLOR_RGB2HSV)
    return hsv_paveikslelis

In [ ]:
def issaugoti_hsv_paveiksleli(hsv_masyvas, issaugojimo_kelias):
    rgb_masyvas = cv2.cvtColor(hsv_masyvas, cv2.COLOR_HSV2RGB)
    paveikslelis = Image.fromarray(rgb_masyvas) #sukuriamas jpg. objektas is masyvo
    paveikslelis.save(issaugojimo_kelias)

In [ ]:
def konvertuoti_viska_i_hsv(pradinis_kelias, issaugojimo_kelias):
    for aplanko_kelias in ["trenyravimas", "validacija", "testas"]:
        pilno_aplanko_kelias = os.path.join(pradinis_kelias, aplanko_kelias)
        naujo_aplanko_kelias = os.path.join(issaugojimo_kelias, aplanko_kelias)

        for klases_kelias in os.listdir(pilno_aplanko_kelias):
            pradzios_klases_aplanko_kelias= os.path.join(pilno_aplanko_kelias, klases_kelias)
            naujos_klases_aplanko_kelias = os.path.join(naujo_aplanko_kelias, klases_kelias)
            os.makedirs(naujos_klases_aplanko_kelias, exist_ok=True) # jei aplankas jau sukurtas eis prie kito veiksmo jei nera sukurs 

            for failo_pavadinimo_kelias in os.listdir(pradzios_klases_aplanko_kelias):
                if failo_pavadinimo_kelias.lower().endswith(('.jpg', '.png', '.jpeg')):
                    pilnas_failo_kelias = os.path.join(pradzios_klases_aplanko_kelias, failo_pavadinimo_kelias)
                    hsv_paveikslelis = konvertuoti_i_hsv(pilnas_failo_kelias)
                    pilnas_kelias_issaugojimui = os.path.join(naujos_klases_aplanko_kelias, failo_pavadinimo_kelias)
                    issaugoti_hsv_paveiksleli(hsv_paveikslelis, pilnas_kelias_issaugojimui)

In [58]:
pradinis_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys"
issaugojimo_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys_hsv"
konvertuoti_viska_i_hsv(pradinis_kelias, issaugojimo_kelias)

In [59]:
dydis = (128, 128)
batch_size = 32
SEED = 42

In [68]:
# 2. TRAIN: įkeliame HSV duomenis
train_ds_hsv_raw = tf.keras.utils.image_dataset_from_directory(
    os.path.join(issaugojimo_kelias, "trenyravimas"),
    image_size=dydis,
    batch_size=batch_size,
    label_mode="categorical",
    seed=SEED,
    shuffle=True
)

# 3. Pasiimam klasių pavadinimus iš train rinkinio (vieną kartą)
klasiu_pavadinimai = train_ds_hsv_raw.class_names
klasiu_skaicius = len(klasiu_pavadinimai)
print("HSV klasės:", klasiu_pavadinimai)

# 4. VAL ir TEST: įkeliam be shuffle
val_ds_hsv_raw = tf.keras.utils.image_dataset_from_directory(
    os.path.join(issaugojimo_kelias, "validacija"),
    image_size=dydis,
    batch_size=batch_size,
    label_mode="categorical",
    seed=SEED,
    shuffle=False
)

test_ds_hsv_raw = tf.keras.utils.image_dataset_from_directory(
    os.path.join(issaugojimo_kelias, "testas"),
    image_size=dydis,
    batch_size=batch_size,
    label_mode="categorical",
    seed=SEED,
    shuffle=False
)

# 5. Sukuriam normalizavimo sluoksnį (pikseliai 0–1)
normalizavimas = tf.keras.layers.Rescaling(1./255)

# 6. Taikom normalizavimą visiems trims rinkiniams
train_ds_hsv = train_ds_hsv_raw.map(lambda x, y: (normalizavimas(x), y))
val_ds_hsv = val_ds_hsv_raw.map(lambda x, y: (normalizavimas(x), y))
test_ds_hsv = test_ds_hsv_raw.map(lambda x, y: (normalizavimas(x), y))

Found 4500 files belonging to 6 classes.
HSV klasės: ['Tomato___Bacterial_spot', 'Tomato___Late_blight', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___healthy']
Found 900 files belonging to 6 classes.
Found 600 files belonging to 6 classes.


In [69]:
def sukurti_pagerinta_cnn(ivesties_forma, klasiu_skaicius, learning_rate=0.001):
    modelis = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=ivesties_forma),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(klasiu_skaicius, activation='softmax')
    ])

    optimizieris = keras.optimizers.Adam(learning_rate=learning_rate)

    modelis.compile(
        optimizer=optimizieris,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return modelis


In [70]:
modelis = sukurti_pagerinta_cnn((128, 128, 3), klasiu_skaicius)

c:\Users\Vartotojas\Desktop\POMIDORAI\pomidorai_repositorija\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [72]:
from tensorflow.keras.callbacks import EarlyStopping

# Sustabdymas, jei val_loss negerėja 4 epohas iš eilės
sustabdymas = EarlyStopping(
    monitor='val_loss',
    patience=4,
    restore_best_weights=True
)

# Modelio treniravimas
history = modelis.fit(
    train_ds_hsv,
    validation_data=val_ds_hsv,
    epochs=40,
    callbacks=[sustabdymas]
)

nuostolis, tikslumas = modelis.evaluate(test_ds_hsv, verbose=1)

print(f"Testo nuostolis (loss): {nuostolis:.4f}")
print(f"Testo tikslumas (accuracy): {tikslumas:.4f}")

Epoch 1/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 128s 902ms/step - accuracy: 0.5271 - loss: 5.5705 - val_accuracy: 0.1678 - val_loss: 27.3386
Epoch 2/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 115s 818ms/step - accuracy: 0.6241 - loss: 1.0697 - val_accuracy: 0.2667 - val_loss: 32.2764
Epoch 3/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 109s 773ms/step - accuracy: 0.6858 - loss: 0.9240 - val_accuracy: 0.5267 - val_loss: 2.5712
Epoch 4/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 122s 626ms/step - accuracy: 0.7262 - loss: 0.8225 - val_accuracy: 0.2733 - val_loss: 21.0419
Epoch 5/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 139s 605ms/step - accuracy: 0.7530 - loss: 0.7747 - val_accuracy: 0.4944 - val_loss: 2.9727
Epoch 6/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 84s 597ms/step - accuracy: 0.7919 - loss: 0.6820 - val_accuracy: 0.7067 - val_loss: 2.0633
Epoch 7/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 86s 611ms/step - accuracy: 0.8270 - loss: 0.5272 - val_accuracy: 0.8444 - val_loss: 0.5910
Epoch 8/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 84s 595ms/step - accuracy: 0.8244 -